In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('car-price_data.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [3]:
sel_df = df[['Make', 'Model','Year','Engine HP','Engine Cylinders','Transmission Type','Vehicle Style','highway MPG','city mpg','MSRP']]

sel_df.columns = sel_df.columns.str.replace(' ', '_').str.lower()

string_columns = list(sel_df.dtypes[sel_df.dtypes == 'object'].index)

for col in string_columns:
    sel_df[col] = sel_df[col].str.lower().str.replace(' ', '_')

sel_df.head()

C:\Users\ahmed\AppData\Local\Temp\ipykernel_13688\1493954606.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df[col] = sel_df[col].str.lower().str.replace(' ', '_')


,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,msrp
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,19,46135
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,19,40650
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,20,36350
3,bmw,1_series,2011,230.0,6.0,manual,coupe,28,18,29450
4,bmw,1_series,2011,230.0,6.0,manual,convertible,28,18,34500


In [4]:
sel_df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [5]:
sel_df = sel_df.fillna(0)
sel_df.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
msrp                 0
dtype: int64

In [6]:
sel_df = sel_df.rename(columns={'msrp': 'price'})
sel_df['transmission_type'].mode()

0    automatic
Name: transmission_type, dtype: object

In [7]:
corr_matrix =  sel_df[['engine_hp','year','engine_cylinders','highway_mpg','city_mpg']].corr()
corr_matrix

,engine_hp,year,engine_cylinders,highway_mpg,city_mpg
engine_hp,1.000000,0.338714,0.774851,-0.415707,-0.424918
year,0.338714,1.000000,-0.040708,0.258240,0.198171
engine_cylinders,0.774851,-0.040708,1.000000,-0.614541,-0.587306
highway_mpg,-0.415707,0.258240,-0.614541,1.000000,0.886829
city_mpg,-0.424918,0.198171,-0.587306,0.886829,1.000000


In [8]:
sel_df['above_average'] = (sel_df.price > sel_df.price.mean()).astype('int')
sel_df.head().T

,0,1,2,3,4
make,bmw,bmw,bmw,bmw,bmw
model,1_series_m,1_series,1_series,1_series,1_series
year,2011,2011,2011,2011,2011
engine_hp,335.0,300.0,300.0,230.0,230.0
engine_cylinders,6.0,6.0,6.0,6.0,6.0
transmission_type,manual,manual,manual,manual,manual
vehicle_style,coupe,convertible,coupe,coupe,convertible
highway_mpg,26,28,28,28,28
city_mpg,19,19,20,18,18
price,46135,40650,36350,29450,34500


In [9]:
df_train_full, df_test = train_test_split(sel_df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [10]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

In [11]:
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

In [12]:
df_train_full = df_train_full.reset_index(drop=True)

df_train_full.above_average.value_counts(normalize=True)

above_average
0    0.723219
1    0.276781
Name: proportion, dtype: float64

In [13]:
global_mean = df_train_full.above_average.mean()
round(global_mean, 3)

0.277

In [14]:
df_train_full.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
above_average          int32
dtype: object

In [15]:
numerical = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']

In [16]:
categorical = ['make', 'model','transmission_type', 'vehicle_style']

In [17]:
df_train_full[categorical].nunique()

make                  48
model                902
transmission_type      5
vehicle_style         16
dtype: int64

In [18]:
from sklearn.metrics import mutual_info_score

def mutual_info_price_score(series):
    return mutual_info_score(series, df_train_full.above_average)

In [19]:
mi = df_train_full[categorical].apply(mutual_info_price_score)
mi.sort_values(ascending=False).round(2)

model                0.46
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

In [20]:
df_train_full[numerical].corrwith(df_train_full.above_average)

year                0.318753
engine_hp           0.660670
engine_cylinders    0.453162
highway_mpg        -0.134484
city_mpg           -0.157912
dtype: float64

One Hot Encoding

In [21]:
from sklearn.feature_extraction import DictVectorizer

In [22]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train[categorical+numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
dv.feature_names_

['city_mpg',
 'engine_cylinders',
 'engine_hp',
 'highway_mpg',
 'make=acura',
 'make=alfa_romeo',
 'make=aston_martin',
 'make=audi',
 'make=bentley',
 'make=bmw',
 'make=bugatti',
 'make=buick',
 'make=cadillac',
 'make=chevrolet',
 'make=chrysler',
 'make=dodge',
 'make=ferrari',
 'make=fiat',
 'make=ford',
 'make=genesis',
 'make=gmc',
 'make=honda',
 'make=hummer',
 'make=hyundai',
 'make=infiniti',
 'make=kia',
 'make=lamborghini',
 'make=land_rover',
 'make=lexus',
 'make=lincoln',
 'make=lotus',
 'make=maserati',
 'make=maybach',
 'make=mazda',
 'make=mclaren',
 'make=mercedes-benz',
 'make=mitsubishi',
 'make=nissan',
 'make=oldsmobile',
 'make=plymouth',
 'make=pontiac',
 'make=porsche',
 'make=rolls-royce',
 'make=saab',
 'make=scion',
 'make=spyker',
 'make=subaru',
 'make=suzuki',
 'make=tesla',
 'make=toyota',
 'make=volkswagen',
 'make=volvo',
 'model=100',
 'model=124_spider',
 'model=190-class',
 'model=1_series',
 'model=2',
 'model=200',
 'model=200sx',
 'model=240',

In [24]:
X_train = pd.DataFrame(X_train, columns=dv.feature_names_)
X_val = pd.DataFrame(X_val, columns=dv.feature_names_)

In [25]:
train_dicts[0]

{'make': 'mitsubishi',
 'model': 'endeavor',
 'transmission_type': 'automatic',
 'vehicle_style': '4dr_suv',
 'year': 2011,
 'engine_hp': 225.0,
 'engine_cylinders': 6.0,
 'highway_mpg': 19,
 'city_mpg': 15}

Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [27]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

model.fit(X_train, y_train)
accuracy_with_all_features = accuracy_score(y_val, model.predict(X_val))
print(accuracy_with_all_features)

0.9395719681074276


In [28]:
df_train[categorical+numerical].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7148 entries, 0 to 7147
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               7148 non-null   object 
 1   model              7148 non-null   object 
 2   transmission_type  7148 non-null   object 
 3   vehicle_style      7148 non-null   object 
 4   year               7148 non-null   int64  
 5   engine_hp          7148 non-null   float64
 6   engine_cylinders   7148 non-null   float64
 7   highway_mpg        7148 non-null   int64  
 8   city_mpg           7148 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 502.7+ KB


In [29]:
all_features = df_train[categorical+numerical]
val_all_features = df_val[categorical+numerical]

In [30]:

least_useful_feature = None
smallest_difference = float('inf')

for feature in all_features.columns:
    X_train_subset = all_features.drop(feature, axis=1)
    X_val_subset = val_all_features.drop(feature, axis=1)
    
    train_dicts = X_train_subset.to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    val_dicts = X_val_subset.to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    
    model.fit(X_train, y_train)
    accuracy_without_feature = accuracy_score(y_val, model.predict(X_val))

    difference = abs(accuracy_with_all_features - accuracy_without_feature)
    print(feature, difference)
    if difference < smallest_difference:
        smallest_difference = difference
        least_useful_feature = feature

least_useful_feature

make 0.009651699538396952
model 0.019303399076793903
transmission_type 0.006294586655476331
vehicle_style 0.007553503986571508
year 0.008812421317666796
engine_hp 0.01552664708350815
engine_cylinders 0.006294586655476331
highway_mpg 0.007133864876206486
city_mpg 0.007133864876206486


'transmission_type'

In [31]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [32]:
y_train_p = np.log1p(df_train.price.values)
y_val_p = np.log1p(df_val.price.values)
y_test_p = np.log1p(df_test.price.values)

In [33]:
for a in [0, 0.01, 0.1, 1, 10]:
    rmse_val = 0
    model_r = Ridge(alpha=a,solver='sag', random_state=42)
    model_r.fit(X_train, y_train_p)
    y_pred = model_r.predict(X_val)
    rmse_val = mean_squared_error(y_val_p, y_pred, squared=False)
    print(a, round(rmse_val,3))

c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0 0.49


c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.01 0.49
